In [84]:
import glob
import pandas as pd
import numpy as np
import re

In [85]:
csv_files = glob.glob('./*.csv')
dataframes=[]
col_names_list = []
for x in csv_files:
    dataframes.append(pd.read_csv(x, low_memory = False))
    col_names_list.append(list(dataframes[-1].columns))


In [86]:
#checks to see if the df has anything to signify country
[[csv_files[y],sum([bool(re.search('ountry',x)) for x in col_names_list[y]])>0] for y in range(len(col_names_list))]

[['.\\city_temperature.csv', True],
 ['.\\EdStatsData.csv', True],
 ['.\\global-city-population-estimates-CITIES-OVER-300K.csv', True],
 ['.\\Qol_data.csv', False],
 ['.\\WPP2019_PopulationByAgeSex_Medium.csv', False],
 ['.\\WRP national data.csv', False]]

In [87]:
Temp_df = dataframes[0].copy()
Ed_df = dataframes[1].copy()
City_pop_df = dataframes[2].copy()
QoL_df = dataframes[3].copy()
Age_df = dataframes[4].copy()
Rel_df = dataframes[5].copy()

In [88]:
for idx in range(6):
    names = col_names_list[idx]
    print(idx)
    print('csv file:' + csv_files[idx])
    for name in names:
        if bool(re.search('ountry',name)):
                print('column name:' + name + '\n','first entry:' + str(dataframes[idx][name][0]) + '\n')

0
csv file:.\city_temperature.csv
column name:Country
 first entry:Algeria

1
csv file:.\EdStatsData.csv
column name:Country Name
 first entry:Arab World

column name:Country Code
 first entry:ARB

2
csv file:.\global-city-population-estimates-CITIES-OVER-300K.csv
column name:Country Code
 first entry:392

column name:Country or area
 first entry:Japan

3
csv file:.\Qol_data.csv
4
csv file:.\WPP2019_PopulationByAgeSex_Medium.csv
5
csv file:.\WRP national data.csv


In [89]:
QoL_df.head()

,year,City,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index
0,2012-Q1,"Berlin, Germany",215.25,140.62,77.34,64.81,89.06,5.74,23.00,40.00
1,2012-Q1,"Zurich, Switzerland",213.18,136.31,83.85,77.78,166.07,8.27,27.33,26.50
2,2012-Q1,"Edmonton, Canada",204.86,124.69,58.98,78.89,112.10,3.59,35.33,21.25
3,2012-Q1,"Perth, Australia",196.00,118.34,48.75,87.04,142.18,5.24,27.00,15.00
4,2012-Q1,"Calgary, Canada",190.78,124.25,71.88,70.42,111.71,4.58,62.00,25.08


In [90]:
QoL_df['year'].unique()

array(['2012-Q1', '2013-Q1', '2014', '2014-mid', '2015', '2015-mid',
       '2016', '2016-mid', '2017', '2017-mid', '2018', '2018-mid', '2019',
       '2019-mid', '2020', '2020-mid', '2021', '2021-mid'], dtype=object)

In [91]:
city_list = list(dataframes[3].copy()['City'].str.split(','))
country_list = []
city_list_df = []
for city in city_list:
    country_list = city[-1]
    city_list_df = city[0]
QoL_df['Country'] = country_list
QoL_df['City'] = city_list_df
QoL_df.head()

,year,City,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Country
0,2012-Q1,Tehran,215.25,140.62,77.34,64.81,89.06,5.74,23.00,40.00,Iran
1,2012-Q1,Tehran,213.18,136.31,83.85,77.78,166.07,8.27,27.33,26.50,Iran
2,2012-Q1,Tehran,204.86,124.69,58.98,78.89,112.10,3.59,35.33,21.25,Iran
3,2012-Q1,Tehran,196.00,118.34,48.75,87.04,142.18,5.24,27.00,15.00,Iran
4,2012-Q1,Tehran,190.78,124.25,71.88,70.42,111.71,4.58,62.00,25.08,Iran


In [92]:
dataframes[5]['name'].unique()

array(['USA', 'CAN', 'BHM', 'CUB', 'HAI', 'DOM', 'JAM', 'TRI', 'BAR',
       'DMA', 'GRN', 'SLU', 'SVG', 'AAB', 'SKN', 'MEX', 'BLZ', 'GUA',
       'HON', 'SAL', 'NIC', 'COS', 'PAN', 'COL', 'VEN', 'GUY', 'SUR',
       'ECU', 'PER', 'BRA', 'BOL', 'PAR', 'CHL', 'ARG', 'URU', 'UKG',
       'IRE', 'NTH', 'BEL', 'LUX', 'FRN', 'MNC', 'LIE', 'SWZ', 'SPN',
       'AND', 'POR', 'GMY', 'GFR', 'GDR', 'POL', 'AUS', 'HUN', 'CZE',
       'CZR', 'SLO', 'ITA', 'SNM', 'MLT', 'ALB', 'MNG', 'MAC', 'CRO',
       'YUG', 'BOS', 'KOS', 'SLV', 'GRC', 'CYP', 'BUL', 'MLD', 'ROM',
       'RUS', 'EST', 'LAT', 'LIT', 'UKR', 'BLR', 'ARM', 'GRG', 'AZE',
       'FIN', 'SWD', 'NOR', 'DEN', 'ICE', 'CAP', 'STP', 'GNB', 'EQG',
       'GAM', 'MLI', 'SEN', 'BEN', 'MAA', 'NIR', 'CDI', 'GUI', 'BFO',
       'LBR', 'SIE', 'GHA', 'TOG', 'CAO', 'NIG', 'GAB', 'CEN', 'CHA',
       'CON', 'DRC', 'UGA', 'KEN', 'TAZ', 'BUI', 'RWA', 'SOM', 'DJI',
       'ETH', 'ERI', 'ANG', 'MZM', 'ZAM', 'ZIM', 'MAW', 'SAF', 'NAM',
       'LES', 'BOT',

In [93]:
Age_df.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,AgeGrp,AgeGrpStart,AgeGrpSpan,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,0-4,0,5,630.044,661.578,1291.622
1,4,Afghanistan,2,Medium,1950,1950.5,5-9,5,5,516.206,487.335,1003.541
2,4,Afghanistan,2,Medium,1950,1950.5,10-14,10,5,461.378,423.326,884.704
3,4,Afghanistan,2,Medium,1950,1950.5,15-19,15,5,414.369,369.363,783.732
4,4,Afghanistan,2,Medium,1950,1950.5,20-24,20,5,374.109,318.392,692.501


In [100]:
Age_df['Country'] = Age_df['Location']
Age_df_m = pd.DataFrame(Age_df[['Location', 'Time', 'PopTotal']].groupby(by = ['Location','Time'],as_index = False).sum())
Age_df_m = Age_df.merge(Age_df_m, on = ['Location', 'Time'], how = 'left')
Age_df_m['AgeGrpPct'] = 100* Age_df_m['PopTotal_x']/Age_df_m['PopTotal_y']
Age_df_m = Age_df_m[['Country', 'Time', 'AgeGrp','AgeGrpPct']]
Age_df = Age_df_m.copy()

In [95]:
Age_df_m['Time'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026,
       2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037,
       2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048,
       2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059,
       2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070,
       2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081,
       2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092,
       2093, 2094, 2095, 2096, 2097, 2098, 2099, 21

In [96]:
print(Rel_df.head())
col = Rel_df.columns
print(col)

   year  state name  chrstprot  chrstcat  chrstorth  chrstang  chrstothr  \
0  1945      2  USA   66069671  38716742    1121898   2400000    1956807   
1  1950      2  USA   73090083  42635882    3045420   3045420    1177214   
2  1955      2  USA   79294628  46402368    3454916   2572767    2277091   
3  1960      2  USA   90692928  50587880    3334535   2710065    2908939   
4  1965      2  USA   94165803  64761783    4792868   2822149     973155   

    chrstgen  judorth  ...  othrgenpct  sumreligpct   total  dualrelig  \
0  110265118   821489  ...      0.0039       0.9961  1.0000          0   
1  122994019  1078078  ...      0.0041       0.9959  1.0000          0   
2  134001770   944000  ...      0.0193       0.9807  0.9999          0   
3  150234347   973500  ...      0.0076       0.9924  0.9999          0   
4  167515758   991200  ...      0.0030       0.9970  1.0001          0   

   datatype  sourcereliab  recreliab  reliabilevel  Version  sourcecode  
0        34             

In [99]:
Rel_df = dataframes[5].copy()[dataframes[5].copy()['year'] == 2010].reset_index(drop = True)
col = Rel_df.columns
col2 = col[0:3:2].append(Rel_df.columns[40:75])
Rel_df = Rel_df[col2]
prefs = ['chrst', 'jud', 'islm','bud', 'zor', 'hin', 'sikh' ,'shnt', 'bah', 'tao', 'jain', 'con', 'syn','anm','non','othr','sum']
Rel_df2 = Rel_df.copy()
for pref in prefs:
    Rel_df2[pref + 'pct'] = np.zeros(len(Rel_df2))
    for name in col2:
        if name[0:len(pref)] == pref:
            Rel_df2[pref + 'pct'] = Rel_df2[pref + 'pct'] + Rel_df2[name]
            Rel_df2 = Rel_df2.drop(name, axis = 1)
Rel_df = Rel_df2.copy()
Rel_df.head()

,year,name,chrstpct,judpct,islmpct,budpct,zorpct,hinpct,sikhpct,shntpct,bahpct,taopct,jainpct,conpct,synpct,anmpct,nonpct,othrpct,sumpct
0,2010,USA,1.4908,0.0381,0.0180,0.0218,0.0053,0.0057,0.0013,0.0005,0.0015,0.0000,0.0003,0.0003,0.0026,0.0057,0.1900,0.0025,0.0
1,2010,CAN,1.5322,0.0199,0.0388,0.0388,0.0002,0.0080,0.0080,0.0000,0.0005,0.0001,0.0001,0.0001,0.0008,0.0021,0.1643,0.0010,0.0
2,2010,BHM,1.9320,0.0020,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0003,0.0000,0.0000,0.0000,0.0032,0.0290,0.0005,0.0
3,2010,CUB,1.3178,0.0002,0.0014,0.0000,0.0000,0.0022,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5000,0.0000,0.1315,0.0000,0.0
4,2010,HAI,1.6400,0.0000,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,0.0009,0.0000,0.0000,0.0000,0.4500,0.0000,0.1000,0.0000,0.0


In [101]:
Temp_df.head()

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [166]:
for x in dataframes[1]['Indicator Name'].unique():
    print(x)

Adjusted net enrolment rate, lower secondary, both sexes (%)
Adjusted net enrolment rate, lower secondary, female (%)
Adjusted net enrolment rate, lower secondary, gender parity index (GPI)
Adjusted net enrolment rate, lower secondary, male (%)
Adjusted net enrolment rate, primary, both sexes (%)
Adjusted net enrolment rate, primary, female (%)
Adjusted net enrolment rate, primary, gender parity index (GPI)
Adjusted net enrolment rate, primary, male (%)
Adjusted net enrolment rate, upper secondary, both sexes (%)
Adjusted net enrolment rate, upper secondary, female (%)
Adjusted net enrolment rate, upper secondary, gender parity index (GPI)
Adjusted net enrolment rate, upper secondary, male (%)
Adjusted net intake rate to Grade 1 of primary education, both sexes (%)
Adjusted net intake rate to Grade 1 of primary education, female (%)
Adjusted net intake rate to Grade 1 of primary education, gender parity index (GPI)
Adjusted net intake rate to Grade 1 of primary education, male (%)
Adul

Population, ages 7-13, male
Population, ages 7-13, total
Population, ages 7-9, female
Population, ages 7-9, male
Population, ages 7-9, total
Population, female
Population, female (% of total)
Population, male
Population, male (% of total)
Population, total
Prevalence of HIV, total (% of population ages 15-49)
Primary completion rate, both sexes (%)
Primary completion rate, female (%)
Primary completion rate, gender parity index (GPI)
Primary completion rate, male (%)
Pupil/qualified teacher ratio in lower secondary (headcount basis)
Pupil/qualified teacher ratio in pre-primary education (headcount basis)
Pupil/qualified teacher ratio in primary education (headcount basis)
Pupil/qualified teacher ratio in secondary (headcount basis)
Pupil/qualified teacher ratio in upper secondary (headcount basis)
Pupil/trained teacher ratio in lower secondary education (headcount basis)
Pupil/trained teacher ratio in pre-primary education (headcount basis)
Pupil/trained teacher ratio in primary educat

In [180]:
Ed_df = dataframes[1].copy()
education_types = []
for x in Ed_df['Indicator Name'].unique():
    if (x.find('Percentage')>0):
        education_types.append(x)

Ed_df = Ed_df[Ed_df['Indicator Name'].isin(education_types)]
col = [colm for colm in Ed_df.columns[4:-1] if (int(colm) > 2011 and int(colm) < 2022)]
col.extend(Ed_df.columns[:3])
Ed_df = Ed_df[col].reset_index(drop = True)
Ed_df = Ed_df[np.logical_or.reduce([np.array(Ed_df[x].isna()) for x in ['2012','2013','2014','2015','2016','2017','2020']])].reset_index(drop = True)
Ed_df.head()

,2012,2013,2014,2015,2016,2017,2020,Country Name,Country Code,Indicator Name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arab World,ARB,Africa Dataset: Percentage of female newly rec...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arab World,ARB,Africa Dataset: Percentage of female newly rec...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arab World,ARB,Africa Dataset: Percentage of female newly rec...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arab World,ARB,Africa Dataset: Percentage of female teachers ...
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Arab World,ARB,Africa Dataset: Percentage of female teachers ...


In [112]:
City_pop_df.head()

,Country Code,Country or area,City Code,Urban Agglomeration,Note,Latitude,Longitude,1950,1955,1960,...,1985,1990,1995,2000,2005,2010,2015,2020,2025,2030
0,392,Japan,21671,Tokyo,325.0,35.689500,139.691710,"11,275","13,713","16,679",...,"30,304","32,530","33,587","34,450","35,622","36,834","38,001","38,323","37,876","37,190"
1,356,India,21228,Delhi,318.0,28.666670,77.216670,"1,369","1,782","2,283",...,"7,325","9,726","12,407","15,732","18,670","21,935","25,703","29,348","32,727","36,060"
2,156,China,20656,Shanghai,202.0,31.220000,121.460000,"4,301","5,846","6,820",...,"6,847","7,823","10,450","13,959","16,763","19,980","23,741","27,137","29,442","30,751"
3,76,Brazil,20287,São Paulo,NaN,-23.550000,-46.640000,"2,334","3,044","3,970",...,"13,395","14,776","15,913","17,014","18,288","19,660","21,066","22,119","22,899","23,444"
4,356,India,21206,Mumbai (Bombay),NaN,19.073975,72.880838,"2,857","3,432","4,060",...,"10,391","12,436","14,310","16,367","17,891","19,422","21,043","22,838","25,207","27,797"


In [ ]:
QoL_df.head()

In [ ]:
Age_df.head()

In [ ]:
Rel_df.head()